In [1]:
import os

In [2]:
%pwd

'd:\\documents\\Code\\InceptionBD\\End-to-End-Deep-Learning-Chicken-Disease\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\documents\\Code\\InceptionBD\\End-to-End-Deep-Learning-Chicken-Disease'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data_path: Path
    params_ephochs: int
    params_batch_size: int
    params_image_size: list
    params_augmentation: bool

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        model_trainer_config = self.config.model_trainer

        trained_model_path = model_trainer_config.trained_model_path

        training_data_path = os.path.join(
            self.config.data_ingestion.unzip_dir,
            "Chicken-fecal-images"
        )

        model_trainer_config = ModelTrainerConfig(
            root_dir = Path(model_trainer_config.root_dir),
            trained_model_path = Path(trained_model_path),
            updated_base_model_path = Path(self.config.prepare_base_model.updated_base_model_path),
            training_data_path = Path(training_data_path),
            params_ephochs = self.params.EPOCHS,
            params_batch_size = self.params.BATCH_SIZE,
            params_image_size = self.params.IMAGE_SIZE,
            params_augmentation = self.params.AUGMENTATION
        )

        return model_trainer_config

In [2]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from cnnClassifier import logger

In [ ]:
class Model_Trainer:
    def __init__(self, TrainingConfig: ModelTrainerConfig):
        self.config = TrainingConfig
    

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)


    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale =1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data_path,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory= self.config.training_data_path,
            subset = "training",
            shuffle = True, # to ensure generalization
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        logger.info(f"Trained Model saved at : {path}")

    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size

        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_ephochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )

        self.save_model(
            self.config.trained_model_path,
            self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_model_trainer_config()
    training = Model_Trainer(training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2025-11-03 11:13:35,063: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-03 11:13:35,069: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-03 11:13:35,071: INFO: common: created directory at: artifacts]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
19/19 [==============================] - 77s 4s/step - loss: 14.6108 - accuracy: 0.5101 - val_loss: 3.8214 - val_accuracy: 0.4375
